# 🏦 AI Search + Agent Service: Banking Products Catalog 💳

Welcome to our **AI Search + AI Agent** tutorial for financial services, where we'll:

1. **Create** an Azure AI Search index with banking products data (loans, credit cards, accounts)
2. **Demonstrate** semantic search functionality with financial product queries
3. **Create** an AI agent with banking knowledge and regulatory disclaimers
4. **Show** how to have conversations about banking products and recommendations

## 🏥 Financial & Regulatory Disclaimer
> **This notebook is for general demonstration and educational purposes, NOT a substitute for professional financial advice.**
> Always consult with licensed financial advisors for personalized recommendations.

## Prerequisites
1. An **Azure AI Search** resource (formerly "Cognitive Search"), provisioned in your Microsoft Foundry project.

## What You'll Learn
- ✅ **Azure AI Search**: Create indexes for financial products, upload documents, perform semantic searches
- ✅ **Agent Service**: Create AI agents with banking product knowledge
- ✅ **Integration Patterns**: Connect search functionality with AI agents for product recommendations
- ✅ **Best Practices**: Error handling, resource cleanup, financial disclaimers

## High-Level Flow
We'll do the following:
1. **Create** an AI Search index with sample banking products data.
2. **Upload** documents (credit cards, loans, accounts) to the index.
3. **Verify** search functionality with test queries.
4. **Create** an AI agent with banking product expertise and proper disclaimers.
5. **Test** agent conversations about financial products.
6. **Clean up** resources responsibly.
 


## 🔐 Authentication Setup

Before running the next cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login --use-device-code
```

This will provide you with a device code and URL to authenticate in your browser, which is useful for:
- Remote development environments
- Systems without a default browser
- Corporate environments with strict security policies

After successful authentication, you can proceed with the notebook cells below.

## 📋 Prerequisites: Create Azure AI Search Connection

Before proceeding with this tutorial, you need to create an Azure AI Search connection in your AI Foundry project.

> ⚠️ **IMPORTANT**: Create the connection with **API Key** authentication (not AAD/Entra ID) to avoid permission issues with the agent.

### **Steps to Create the Connection:**

1. **Navigate to Microsoft Foundry portal**: https://ai.azure.com/
2. **Go to your project** → **Settings** → **Connections**
3. **Click "New Connection"** → **Azure AI Search**
4. **Provide the following details:**
   - **Connection Name**: Give it a descriptive name (e.g., "my-ai-search")
   - **Authentication**: ⚠️ **Select API Key** (NOT AAD/Entra ID)
   - **Subscription**: Select your Azure subscription
   - **Resource Group**: Choose your resource group
   - **Azure AI Search Resource**: Select your existing search service or create a new one
5. **Click "Add Connection"** to complete the setup
6. **📝 Note down your connection name** - you'll need it in the agent creation cell below!

### **⚠️ Important Notes:**
- The connection must be created **before** running the code below
- **Use API Key authentication** for the connection (AAD/Entra ID auth has permission issues with agents)
- **Write down your connection name** - update the `ai_search_connection_name` variable in the agent creation cell
- If you don't have an Azure AI Search resource, you'll need to create one first

---


## 1. Create & Populate Azure AI Search Index with Banking Products

We'll create a **vector search enabled** index called `bankingproductsindex`, containing financial products with both traditional fields and vector embeddings.

**Key Features:**
- **Vector Field**: `DescriptionVector` stores embeddings for semantic search
- **HNSW Algorithm**: Fast approximate nearest neighbor search
- **Azure OpenAI Vectorizer**: Integrated embedding generation
- **Semantic Understanding**: Searches based on meaning, not just keywords

This enables the agent to understand queries like "low-fee credit card" or "best savings rate" even when those exact words aren't in the product descriptions!

In [ ]:
# Import required Azure libraries
import os
from pathlib import Path
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex, 
    SimpleField, 
    SearchFieldDataType, 
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch
)
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType, PromptAgentDefinition
from openai import AzureOpenAI

# Load environment variables
load_dotenv(Path().parent.parent / '.env')

# Initialize project client with DefaultAzureCredential (uses az login)
credential = DefaultAzureCredential()

project_client = AIProjectClient(
    endpoint=os.environ["AI_FOUNDRY_PROJECT_ENDPOINT"], 
    credential=credential
)

# Get search connection and create clients
search_conn = project_client.connections.get_default(
    connection_type=ConnectionType.AZURE_AI_SEARCH, 
    include_credentials=True
)

# Check if we have a key or need to use AAD authentication
index_name = "bankingproductsindex"

if search_conn.credentials.get('type') == 'AAD':
    # Use the same credential for Azure AI Search (AAD authentication)
    print("🔐 Using Azure AD authentication for AI Search")
    search_credential = credential
else:
    # Use API key if available
    search_credential = AzureKeyCredential(search_conn.credentials['key'])

index_client = SearchIndexClient(
    endpoint=search_conn.target, 
    credential=search_credential
)

search_client = SearchClient(
    endpoint=search_conn.target,
    index_name=index_name,
    credential=search_credential
)

# Initialize Azure OpenAI client for embeddings
openai_client = AzureOpenAI(
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-01",
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT")
)

print(f"✅ Initialized clients for banking products index: {index_name}")
print(f"   Search endpoint: {search_conn.target}")

## Create Search Index Schema for Banking Products

In [ ]:
# Import additional models for semantic search
from azure.search.documents.indexes.models import (
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch
)
from azure.core.exceptions import ResourceNotFoundError
import time

# Define index schema with banking product fields including vector field
embedding_model = os.environ.get("EMBEDDING_MODEL_DEPLOYMENT_NAME", "text-embedding-ada-002")
embedding_dimensions = 1536

if "text-embedding-3-small" in embedding_model:
    embedding_dimensions = 1536
elif "text-embedding-3-large" in embedding_model:
    embedding_dimensions = 3072

def create_banking_index(index_name):
    """Create a vector search index with hybrid search capabilities for banking products."""
    
    # Check if index exists and delete it
    try:
        index_client.get_index(index_name)
        print(f"🗑️ Deleting existing index '{index_name}'...")
        index_client.delete_index(index_name)
        time.sleep(2)
    except ResourceNotFoundError:
        print(f"   Index '{index_name}' does not exist, creating new...")
    
    # Define fields
    fields = [
        SimpleField(name="ProductID", type=SearchFieldDataType.String, key=True),
        SearchableField(name="Name", type=SearchFieldDataType.String, filterable=True),
        SearchableField(name="Category", type=SearchFieldDataType.String, filterable=True, facetable=True),
        SimpleField(name="APR", type=SearchFieldDataType.Double, filterable=True, sortable=True, facetable=True),
        SimpleField(name="AnnualFee", type=SearchFieldDataType.Double, filterable=True, sortable=True),
        SearchableField(name="Description", type=SearchFieldDataType.String),
        SearchField(
            name="DescriptionVector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=embedding_dimensions,
            vector_search_profile_name="myHnswProfile"
        )
    ]

    # Configure vector search with HNSW algorithm and Azure OpenAI vectorizer
    vector_search = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(name="myHnsw")
        ],
        profiles=[
            VectorSearchProfile(
                name="myHnswProfile",
                algorithm_configuration_name="myHnsw",
                vectorizer_name="myOpenAI"
            )
        ],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="myOpenAI",
                kind="azureOpenAI",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=os.environ.get("AZURE_OPENAI_ENDPOINT"),
                    deployment_name=embedding_model,
                    model_name=embedding_model
                )
            )
        ]
    )

    # Configure semantic search
    semantic_config = SemanticConfiguration(
        name="banking-semantic-config",
        prioritized_fields=SemanticPrioritizedFields(
            title_field=SemanticField(field_name="Name"),
            content_fields=[SemanticField(field_name="Description")],
            keywords_fields=[SemanticField(field_name="Category")]
        )
    )

    semantic_search = SemanticSearch(configurations=[semantic_config])

    # Create the index with vector and semantic search
    index = SearchIndex(
        name=index_name, 
        fields=fields, 
        vector_search=vector_search,
        semantic_search=semantic_search
    )
    
    index_client.create_or_update_index(index)
    print(f"✅ Created banking products index: {index_name}")

# Create the index
print("🏗️ Creating Azure AI Search Index...")
print("="*60)
create_banking_index(index_name)
print(f"   Embedding model: {embedding_model}")
print(f"   Embedding dimensions: {embedding_dimensions}")

## Upload Sample Banking Products

In [ ]:
# Function to generate embeddings using Azure OpenAI
def generate_embeddings(texts, batch_size=10):
    """Generate embeddings for the given texts using Azure OpenAI."""
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        response = openai_client.embeddings.create(
            input=batch,
            model=embedding_model
        )
        embeddings.extend([item.embedding for item in response.data])
    return embeddings

def upload_banking_products(index_name, products):
    """Upload banking products with embeddings to search index."""
    # Create search client for uploads
    upload_client = SearchClient(
        endpoint=search_conn.target,
        index_name=index_name,
        credential=search_credential
    )
    
    # Generate embeddings for all products
    texts = [f"{p['Name']} {p['Category']} {p['Description']} APR: {p['APR']}%" for p in products]
    print(f"   ⏳ Generating {len(texts)} embeddings...")
    embeddings = generate_embeddings(texts)
    
    # Add embeddings to products
    for i, product in enumerate(products):
        product["DescriptionVector"] = embeddings[i]
        print(f"   ✓ Generated embedding for: {product['Name']}")
    
    # Upload documents
    result = upload_client.upload_documents(documents=products)
    return len(products)

# Sample banking products - credit cards, loans, and accounts
sample_products = [
    {
        "ProductID": "CC001",
        "Name": "Premier Rewards Credit Card",
        "Category": "Credit Card", 
        "APR": 18.99,
        "AnnualFee": 95.0,
        "Description": "Premium rewards credit card with 3% cash back on dining and travel, 2% on groceries, and 1% on all other purchases. Includes travel insurance and airport lounge access."
    },
    {
        "ProductID": "CC002",
        "Name": "No-Fee Cash Back Card",
        "Category": "Credit Card",
        "APR": 21.99,
        "AnnualFee": 0.0,
        "Description": "No annual fee credit card with 1.5% unlimited cash back on all purchases. Perfect for everyday spending without worrying about annual fees."
    },
    {
        "ProductID": "MORT001",
        "Name": "30-Year Fixed Mortgage",
        "Category": "Mortgage",
        "APR": 6.5,
        "AnnualFee": 0.0,
        "Description": "Traditional 30-year fixed-rate mortgage with predictable monthly payments. Minimum 620 credit score required. Down payment as low as 3%."
    },
    {
        "ProductID": "MORT002",
        "Name": "15-Year Fixed Mortgage",
        "Category": "Mortgage",
        "APR": 5.75,
        "AnnualFee": 0.0,
        "Description": "Shorter-term mortgage with lower interest rate. Build equity faster with higher monthly payments. Ideal for borrowers who want to pay off home quickly."
    },
    {
        "ProductID": "SAV001",
        "Name": "High-Yield Savings Account",
        "Category": "Savings Account",
        "APR": 4.5,
        "AnnualFee": 0.0,
        "Description": "Online savings account with competitive APY. No minimum balance required. FDIC insured up to $250,000. Unlimited transfers to linked checking."
    },
    {
        "ProductID": "AUTO001",
        "Name": "New Auto Loan",
        "Category": "Auto Loan",
        "APR": 6.49,
        "AnnualFee": 0.0,
        "Description": "Competitive rates for new vehicle purchases. Terms from 36 to 84 months. Quick approval process and flexible payment options."
    },
    {
        "ProductID": "PERS001",
        "Name": "Personal Line of Credit",
        "Category": "Personal Loan",
        "APR": 11.99,
        "AnnualFee": 0.0,
        "Description": "Flexible borrowing for unexpected expenses or debt consolidation. Borrow up to $50,000. Only pay interest on what you use."
    }
]

# Upload banking products with embeddings
print("🧠 Generating embeddings and uploading documents...")
print("="*60)
count = upload_banking_products(index_name, sample_products)
print(f"\n{'='*60}")
print(f"✅ Uploaded {count} banking products with embeddings to index: {index_name}")

## 2. Create Banking Product Advisor Agent With Azure AI Search Tool

Configure the official AzureAISearchTool and create an AI agent with direct search integration for banking products.

In [ ]:
# Configure Azure AI Search tool for banking products
from azure.ai.projects.models import (
    AzureAISearchAgentTool,
    AzureAISearchToolResource,
    AISearchIndexResource,
    AzureAISearchQueryType,
)

# Use the new API Key-authenticated connection
ai_search_connection_name = "kdfoundryaisearchqr936x"

# Print connection details
print("🔍 Search Connection Details:")
print(f"   Connection Name: {ai_search_connection_name}")
print(f"   Index Name: {index_name}")

# Configure the AI Search tool - use the new API Key connection
tool = AzureAISearchAgentTool(
    azure_ai_search=AzureAISearchToolResource(
        indexes=[
            AISearchIndexResource(
                project_connection_id=ai_search_connection_name,  # Use new API Key connection
                index_name=index_name,
                query_type=AzureAISearchQueryType.SEMANTIC,
            ),
        ]
    )
)

print(f"\n✅ AI Search Tool configured:")
print(f"   Project Connection: {ai_search_connection_name}")
print(f"   Index: {index_name}")
print(f"   Query Type: SEMANTIC")

# Create the agent following official SDK pattern
agent = project_client.agents.create_version(
    agent_name="banking-product-advisor-agent",
    definition=PromptAgentDefinition(
        model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"],
        instructions="""You are a Banking Product Advisor with access to a live product catalog through Azure AI Search.

**Your Capabilities:**
- Use search to find banking products based on customer needs
- Search by APR, fees, product category, and benefits
- Provide product comparisons and recommendations

**Banking Product Categories:**
- Credit Cards: Rewards, cash back, no annual fee options
- Mortgages: 15-year and 30-year fixed rate options  
- Savings Accounts: High-yield savings options
- Auto Loans: New and used vehicle financing
- Personal Loans: Lines of credit and debt consolidation

**Guidelines:**
- Always search for products when customers ask about specific needs
- Include specific APRs, fees, and features in your responses
- Include financial disclaimers: "This is for informational purposes only. Please consult with a banking representative for personalized advice."
- Always provide citations for answers using the tool and render them as: `【message_idx:search_idx†source】`

Be helpful, professional, and prioritize customer education!""",
        tools=[tool],
    ),
    description="Banking product advisor agent",
)

# Get OpenAI client for conversations
foundry_openai_client = project_client.get_openai_client()

print(f"\n🎉 Created agent: {agent.name} (Version: {agent.version})")
print(f"📝 Agent ID: {agent.id}")

## 3. Test Banking Product Advisor Agent

Test the agent with banking product questions. The agent will automatically use the Azure AI Search tool to find relevant products and provide recommendations.

In [ ]:
# Test the banking product advisor agent
def test_agent_question(question):
    print(f"\n🔍 **Question:** {question}")
    print("=" * 70)
    
    try:
        # Create conversation and send message using new Foundry API
        conversation = foundry_openai_client.conversations.create()
        
        # Get response using the new responses API with agent reference
        response = foundry_openai_client.responses.create(
            conversation=conversation.id,
            extra_body={
                "agent": {
                    "type": "agent_reference",
                    "name": agent.name,
                    "version": agent.version
                }
            },
            input=question
        )
        
        # Get the response content
        if response.output_text:
            print("🤖 **Banking Advisor Response:**")
            print(response.output_text)
        else:
            print(f"❌ No response received")
            
    except Exception as e:
        error_msg = str(e)
        if "Access denied" in error_msg or "permissions" in error_msg:
            print("⚠️ **PERMISSIONS ERROR**")
            print("The agent cannot access the Azure AI Search service.")
            print("\n📋 To fix this, add the following role assignments:")
            print("1. 'Search Index Data Reader' role on the AI Search resource")
            print("2. Assign to your Microsoft Foundry project's managed identity")
            print("\n💡 For this workshop, the vector search already demonstrated the concept!")
        else:
            print(f"❌ Error: {e}")
    
    print("=" * 70)

# Test with banking product questions
test_questions = [
    "I'm looking for a credit card with no annual fee and good cash back rewards. What do you have?",
]

print("🧪 Testing Banking Product Advisor Agent")
print("=" * 70)

for question in test_questions:
    test_agent_question(question)

print("\n📝 KEY CONCEPTS DEMONSTRATED:")
print("   1. Created Azure AI Search index with banking products")
print("   2. Configured semantic search with vector embeddings")
print("   3. Built agent with AzureAISearchAgentTool integration")
print("   4. Vector search queries find products by meaning, not just keywords")
print("\n💡 The vector search results earlier showed the concept works correctly!")

In [ ]:
# Clean up resources
try:
    # Delete agent (correct parameter name is agent_version)
    project_client.agents.delete_version(agent_name=agent.name, agent_version=agent.version)
    print(f"🗑️ Deleted agent: {agent.name} (Version: {agent.version})")
except Exception as e:
    print(f"⚠️ Could not delete agent: {e}")

try:
    # Delete search index
    index_client.delete_index(index_name)
    print(f"🗑️ Deleted search index: {index_name}")
except Exception as e:
    print(f"⚠️ Could not delete index: {e}")

print("\n✅ Cleanup completed!")

# 🎉 Congratulations!

You've successfully completed the **AI Search + Agent Service with Vector Search** tutorial!

## ✅ **What We Accomplished**

1. **🔍 Azure AI Search with Vector Search & Semantic Configuration**
   - Created a search index with **vector embeddings** for semantic search
   - Configured **HNSW algorithm** for efficient vector similarity search
   - Used **Azure OpenAI embeddings** (text-embedding-ada-002) to vectorize product descriptions
   - Implemented **integrated vectorization** with AzureOpenAIVectorizer
   - Added **Semantic Configuration** (required for SEMANTIC query type)
   - Included price information in embeddings for better context understanding

2. **🤖 AI Agent with Semantic Search Capabilities**  
   - Created an agent using **SEMANTIC query type** (powered by vector embeddings)
   - Agent understands semantic meaning and can find products based on:
     - **Categories** ("strength training", "cardio", "flexibility")
     - **Use cases** ("muscle recovery", "home gym", "conditioning")
     - **Concepts** ("affordable", "budget-friendly", "full-body")
     - **Natural language** queries without exact keyword matches
   - Seamless integration with real-time semantic search results

3. **💡 Key Improvements Over Simple Search**
   - **Semantic Understanding**: Finds relevant items based on meaning, not just keywords
   - **Context Awareness**: Understands "muscle recovery" → foam roller, yoga mat
   - **Better Relevance**: Returns results based on semantic similarity
   - **Richer Embeddings**: Combined name, category, description, and price in vector embeddings
   - **Natural Language**: Understands intent from conversational queries

4. **📊 Technical Implementation**
   - **Vector Field**: `DescriptionVector` with 1536 dimensions
   - **Vector Search**: HNSW algorithm for fast approximate nearest neighbor search
   - **Semantic Config**: Required for SEMANTIC query type with Name, Description, Category fields
   - **Manual Vectorization**: Pre-computed embeddings using Azure OpenAI
   - **Official SDK**: Uses `azure-search-documents` and `azure-ai-agents` packages

## 🎯 **How Vector Search Works in This Example**

1. **Indexing Time**:
   - Documents are enriched with text combining all relevant fields + price
   - Azure OpenAI generates 1536-dimensional embeddings
   - Embeddings stored in `DescriptionVector` field
   - HNSW algorithm creates efficient search structure

2. **Query Time**:
   - User asks a natural language question to the agent
   - Agent uses SEMANTIC search (leverages pre-computed vectors)
   - Search returns most semantically similar items
   - Agent formats results with prices and details

## 🔧 **Technical Architecture**

```
User Query (Natural Language)
    ↓
AI Agent (SEMANTIC query type)
    ↓
Azure AI Search Index
    ├─ Traditional fields (Name, Price, Category)
    └─ Vector field (DescriptionVector with embeddings)
    ↓
HNSW Algorithm (similarity search)
    ↓
Semantic Results (most similar items)
    ↓
Agent Response (formatted with context)
```

## 📖 **Key Learnings**

1. **Vector Search Types**:
   - **VECTOR query type**: Requires runtime vectorization (authentication challenges)
   - **SEMANTIC query type**: Uses pre-computed vectors (what we implemented)
   - Both leverage the same vector index and embeddings

2. **Semantic Configuration**:
   - Required for SEMANTIC query type
   - Defines title, content, and keyword fields
   - Improves relevance ranking

3. **Embedding Strategy**:
   - Include all relevant context in text for embedding
   - Price information embedded as text: "Price: $59.99"
   - Enables semantic understanding of price concepts

4. **Best Query Patterns for Semantic Search**:
   - ✅ **Good**: "affordable strength training equipment" (concepts)
   - ✅ **Good**: "equipment for muscle recovery" (use cases)
   - ✅ **Good**: "budget-friendly home gym options" (categories)
   - ❌ **Limited**: "show items under $50" (exact numerical filters)
   - ❌ **Limited**: "what's the cheapest item" (superlatives requiring sorting)

5. **When to Use What**:
   - **Semantic Search**: Best for understanding intent, categories, concepts
   - **Filters**: Best for exact numerical ranges (combine with OData: `Price le 100`)
   - **Hybrid**: Combine semantic search + filters for best results

## 🚀 **Next Steps & Improvements**

1. **Add OData Filters**: Combine semantic search with exact price filters
   ```python
   filter="Price le 100"  # Combine with semantic search
   ```

2. **Hybrid Search**: Combine keyword search + vector search
   ```python
   search_text="strength",  # Keyword search
   vector_queries=[vector_query]  # Vector search
   ```

3. **Semantic Ranker**: Enable L2 semantic ranking for better relevance

4. **Multi-field Vectors**: Create separate vectors for different aspects

5. **Query Expansion**: Let agent rephrase queries for better results

## 🎓 **Summary**

**Semantic Search Excels At**:
- Understanding natural language and intent
- Finding items by category, use case, or concept
- Providing contextually relevant results
- Handling synonyms and related terms

**For Exact Filtering, Add**:
- OData filters for price ranges
- Traditional field filters
- Faceting for categorical filtering

## 📚 **Resources**

- [Vector Search Overview](https://learn.microsoft.com/en-us/azure/search/vector-search-overview)
- [Create Vector Index](https://learn.microsoft.com/en-us/azure/search/vector-search-how-to-create-index)
- [Vector Query Guide](https://learn.microsoft.com/en-us/azure/search/vector-search-how-to-query)
- [Azure Search Vector Samples](https://github.com/Azure/azure-search-vector-samples)
- [OData Filters in Azure Search](https://learn.microsoft.com/en-us/azure/search/search-query-odata-filter)

Ready to build production AI applications with Azure AI Search Vector Search + Agent integration! 🚀